In [1]:
import os
import glob
import cv2
import pyrealsense2 as rs
import numpy as np
import matplotlib.pyplot as plt

# Setup

In [2]:
def read_bag_file(file_name):
    pipe = rs.pipeline()
    cfg = rs.config()
    cfg.enable_device_from_file(file_name, repeat_playback=False)
    profile = pipe.start(cfg)
    playback = profile.get_device().as_playback()
    playback.set_real_time(False) # False: no frame drop
    
    # Get the frame shape of the color sensor
    frames = pipe.wait_for_frames()
    color_frame = frames.get_color_frame()
    frame_shape = (color_frame.get_height(), color_frame.get_width())

    return pipe, cfg, profile, playback, frame_shape

In [3]:
# Read the bag file
# file_name = "..\\Dataset\\_realsense\\Data_realsense1\\20240322_123504.bag"
#file_name = "..\\..\\Dataset\\tests\\20240620_113148.bag"
file_name = "..\\bag_records\\me_and_sasa.bag"

pipe, cfg, profile, playback, frame_shape = read_bag_file(file_name)
duration = playback.get_duration()
print("Video duration: ", duration)
print("Frame shape: ", frame_shape)

Video duration:  0:00:43.503743
Frame shape:  (480, 848)


# Visualize the data

In [4]:
def post_process_depth_frame(depth_frame):
    # Post processing possible only on the depth_frame
    assert (depth_frame.is_depth_frame())

    decimation_magnitude = 1.0
    spatial_magnitude = 2.0
    spatial_smooth_alpha = 0.5
    spatial_smooth_delta = 20
    temporal_smooth_alpha = 0.4
    temporal_smooth_delta = 20

    # Available filters
    decimation_filter = rs.decimation_filter()
    depth_to_disparity = rs.disparity_transform(True)
    spatial_filter = rs.spatial_filter()
    temporal_filter = rs.temporal_filter()
    disparity_to_depth = rs.disparity_transform(False)

    # Options for hole_filling_filter
    # 0 - fill_from_left - Use the value from the left neighbor pixel to fill the hole
    # 1 - farest_from_around - Use the value from the neighboring pixel which is furthest away from the sensor
    # 2 - nearest_from_around - Use the value from the neighboring pixel closest to the sensor
    hole_filling = rs.hole_filling_filter(1)

    # Apply the control parameters for the filters
    decimation_filter.set_option(rs.option.filter_magnitude, decimation_magnitude)
    spatial_filter.set_option(rs.option.filter_magnitude, spatial_magnitude)
    spatial_filter.set_option(rs.option.filter_smooth_alpha, spatial_smooth_alpha)
    spatial_filter.set_option(rs.option.filter_smooth_delta, spatial_smooth_delta)
    temporal_filter.set_option(rs.option.filter_smooth_alpha, temporal_smooth_alpha)
    temporal_filter.set_option(rs.option.filter_smooth_delta, temporal_smooth_delta)

    # Apply the filters
    # Post processing order : https://dev.intelrealsense.com/docs/post-processing-filters
    # Depth Frame >> Decimation Filter >> Depth2Disparity Transform >> Spatial Filter 
    # >> Temporal Filter >> Disparity2Depth Transform >> Hole Filling Filter >> Filtered Depth
    filtered_frame = decimation_filter.process(depth_frame)
    filtered_frame = depth_to_disparity.process(filtered_frame)
    filtered_frame = spatial_filter.process(filtered_frame)
    filtered_frame = temporal_filter.process(filtered_frame)
    filtered_frame = disparity_to_depth.process(filtered_frame)
    filtered_frame = hole_filling.process(filtered_frame)
    
    # Cast to depth_frame so that we can use the get_distance method afterwards
    depth_frame_filtered = filtered_frame.as_depth_frame()

    return depth_frame_filtered

In [5]:
colorizer = rs.colorizer()
value_min = 0.0 # 0 meters
value_max = 1.5 # 1.5 meters
colorizer.set_option(rs.option.min_distance, value_min)
colorizer.set_option(rs.option.max_distance, value_max)

align = rs.align(rs.stream.color)

# Read the full stream
pipe, cfg, profile, playback, frame_shape = read_bag_file(file_name)
num_frames = 0
wait_key = 1

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        # Post process is not included in the BAG file, so we need to apply it
        depth_frame = post_process_depth_frame(aligned_depth_frame)

        # Colorize the depth frame
        depth_color_frame = colorizer.colorize(depth_frame)

        depth_color_image = np.asanyarray(depth_color_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        cv2.imshow("Color Image", color_image)
        cv2.imshow("Depth Image", depth_color_image)

        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press s to write the color_image on disk
        if key == ord('s'):
            cv2.imwrite(".\\images\\color_images\\images\\color_" + str(num_frames) + ".png", color_image)
            cv2.imwrite(".\\images\\depth_images\\depth_" + str(num_frames) + ".png", depth_color_image)

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1

# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop()

print("Total number of frames: ", num_frames)

Total number of frames:  49


# Extract height of the participants

In [6]:
# # Create a mask to keep ony the ROI, under the camera
# mask = np.zeros(frame_shape, dtype=np.uint8)
# center_y, center_x = frame_shape[0] // 2, frame_shape[1] // 2

# # Mask size
# half_height = frame_shape[0] // 3
# top_left = (0, half_height)
# bottom_right = (848, 480)

# cv2.rectangle(mask, top_left, bottom_right, 255, thickness=-1)

# plt.imshow(mask, cmap='gray')
# plt.show()

In [6]:
# Setup the aruco detector
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_ARUCO_MIP_36h12)
aruco_params = cv2.aruco.DetectorParameters()

aruco_params.errorCorrectionRate = 0.2 # default 0.6
aruco_params.polygonalApproxAccuracyRate = 0.05 # default 0.03

detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params) 

In [7]:
camera_height = 2.568 # meters, ground truth

pipe, cfg, profile, playback, frame_shape = read_bag_file(file_name)
num_frames = 0
wait_key = 1

# Data structure to store the a list of height for every aruco marker id
heights = {}

try:
    while True:
        # Get frameset of color and depth
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame
        aligned_frames = align.process(frames)

        # Get aligned frames
        aligned_depth_frame = aligned_frames.get_depth_frame()
        color_frame = aligned_frames.get_color_frame()

        # Validate that both frames are valid
        if not aligned_depth_frame or not color_frame:
            continue

        # Post process is not included in the BAG file, so we need to apply it
        depth_frame = post_process_depth_frame(aligned_depth_frame)

        # Colorize the depth frame
        depth_color_frame = colorizer.colorize(depth_frame)

        depth_color_image = np.asanyarray(depth_color_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Detect the aruco markers
        corners, ids, rejected = detector.detectMarkers(color_image)
        output_image = depth_color_image.copy()
        cv2.aruco.drawDetectedMarkers(output_image, corners, ids)

        for k in range(len(corners)):
            id = ids[k][0]
            c = corners[k][0]

            # Calculate the distance using the center of the aruco marker
            x = int(c[:, 0].sum() / 4)
            y = int(c[:, 1].sum() / 4)
            distance = depth_frame.get_distance(x, y)

            # Calculate the height
            height = camera_height - distance

            # Display the height with an offset to avoid overlap
            text_position_y = 30 + k * 40
            cv2.putText(output_image, "ID: {} Height: {:.2f}m".format(id, height), (10, text_position_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            
            # Store the height for the aruco marker id
            if not id in heights:
                heights[id] = []
                
            heights[id].append(height)

        cv2.imshow("Output", output_image)
        cv2.imshow("Color Image", color_image)
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop()

In [8]:
def remove_outliers(data, m=2):
    # Z-score method
    data = np.array(data)
    mean = np.mean(data)
    std = np.std(data)
    filtered_data = [x for x in data if abs(x - mean) / std < m]
    return filtered_data

def calculate_mean_height(heights):
    mean_heights = {}

    for key, height_list in heights.items():
        filtered_heights = remove_outliers(height_list)
        if filtered_heights:
            mean_heights[key] = np.mean(filtered_heights)

    return mean_heights

# Remove outliers and calculate the mean height for each aruco code id
mean_heights = calculate_mean_height(heights)
print(mean_heights)

{}


# Two streams

In [11]:
colorizer = rs.colorizer()
value_min = 0.0 # 0 meters
value_max = 1.5 # 1.5 meters
colorizer.set_option(rs.option.min_distance, value_min)
colorizer.set_option(rs.option.max_distance, value_max)

align = rs.align(rs.stream.color)

file_name1 = "..\\bag_records\\first.bag"
file_name2 = "..\\bag_records\\second.bag"
pipe1, cfg1, profile1, playback1, frame_shape1 = read_bag_file(file_name1)
pipe2, cfg2, profile2, playback2, frame_shape2 = read_bag_file(file_name2)
num_frames = 0
wait_key = 1

color_images1 = []
color_images2 = []
save_images = False

try:
    while True:
        # Get framesets of color and depth
        frames1 = pipe1.wait_for_frames()
        frames2 = pipe2.wait_for_frames()

        # Align the depth frames to color frames
        aligned_frames1 = align.process(frames1)
        aligned_frames2 = align.process(frames2)

        # Get aligned frames
        aligned_depth_frame1 = aligned_frames1.get_depth_frame()
        color_frame1 = aligned_frames1.get_color_frame()

        aligned_depth_frame2 = aligned_frames2.get_depth_frame()
        color_frame2 = aligned_frames2.get_color_frame()

        # Validate that frames are valid
        if not aligned_depth_frame1 or not color_frame1 or not aligned_depth_frame2 or not color_frame2:
            continue

        # Post process is not included in the BAG file, so we need to apply it
        depth_frame1 = post_process_depth_frame(aligned_depth_frame1)
        depth_frame2 = post_process_depth_frame(aligned_depth_frame2)

        # Colorize the depth frames
        depth_color_frame1 = colorizer.colorize(depth_frame1)
        depth_color_frame2 = colorizer.colorize(depth_frame2)

        depth_color_image1 = np.asanyarray(depth_color_frame1.get_data())
        color_image1 = np.asanyarray(color_frame1.get_data())
        depth_color_image2 = np.asanyarray(depth_color_frame2.get_data())
        color_image2 = np.asanyarray(color_frame2.get_data())

        # Stack the color images horizontally
        color_image1 = cv2.resize(color_image1, (0, 0), fx = 0.7, fy = 0.7)
        color_image2 = cv2.resize(color_image2, (0, 0), fx = 0.7, fy = 0.7)
        color_images = np.hstack((color_image1, color_image2))

        # Stack the depth images horizontally
        depth_color_image1 = cv2.resize(depth_color_image1, (0, 0), fx = 0.7, fy = 0.7)
        depth_color_image2 = cv2.resize(depth_color_image2, (0, 0), fx = 0.7, fy = 0.7)
        depth_images = np.hstack((depth_color_image1, depth_color_image2))

        if save_images:
            color_images1.append(color_image1)
            color_images2.append(color_image2)

        cv2.imshow("Color Images", color_images)
        cv2.imshow("Depth Images", depth_images)

        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break
        
        if key == ord('s'):
            save_images = not save_images

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1

# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe1.stop()
    pipe2.stop()

print("Total number of frames: ", num_frames)

Total number of frames:  696


In [12]:
for image1, image2 in zip(color_images1, color_images2):
    cv2.imshow("Color Image 1", image1)
    cv2.imshow("Color Image 2", image2)
    cv2.waitKey(0)

: 

# Attemping to enhance Aruco codes

In [25]:
""" camera_height = 2.65 # meters => NOT SURE ABOUT THIS VALUE

pipe, cfg, profile, playback, frame_shape = read_bag_file(file_name)
num_frames = 0
wait_key = 1

# Empty the distance_images folder
files = glob.glob('distance_images/*')
for f in files:
    os.remove(f)

# Data structure to store the a list of height for every aruco code id
heights = {}

try:
    while True:
        frames = pipe.wait_for_frames()

        # Align the depth frame to color frame so they have the same shape
        aligned_depth, aligned_color = align_frames(frames)
        color_image = np.asanyarray(aligned_color.get_data())
        depth_color_frame = colorizer.colorize(aligned_depth)
        depth_color_image = np.asanyarray(depth_color_frame.get_data())

        # Apply the mask to the depth image
        depth_color_image = cv2.bitwise_and(depth_color_image, depth_color_image, mask=mask)

        # Convert the image to HSV color space
        hsv_image = cv2.cvtColor(depth_color_image, cv2.COLOR_BGR2HSV)

        # Define the range for red color in HSV
        # Note: Red can appear in two ranges in HSV
        lower_red1 = np.array([0, 70, 50])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([170, 70, 50])
        upper_red2 = np.array([180, 255, 255])

        # Create masks for red color
        red_mask1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
        red_mask2 = cv2.inRange(hsv_image, lower_red2, upper_red2)

        # Combine the two masks
        red_mask = cv2.bitwise_or(red_mask1, red_mask2)

        # Apply the mask to the original image
        result = cv2.bitwise_and(depth_color_image, depth_color_image, mask=red_mask)

        # Convert the result to grayscale and apply thresholding
        gray_result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
        _, thresh_result = cv2.threshold(gray_result, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply the closing morphological operation to fill in holes
        kernel = np.ones((30, 30), np.uint8)
        closed_result = cv2.morphologyEx(thresh_result, cv2.MORPH_CLOSE, kernel)

        # Find contours of the red square
        contours, _ = cv2.findContours(closed_result, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Create a mask to keep only the aruco code in the color image
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 100*100 and area < 200*200:
                x, y, w, h = cv2.boundingRect(contour)  
                arcuo_code_mask = np.zeros(frame_shape, dtype=np.uint8)
                
                # Expand the bounding box by 50 pixels
                sz = (w + h) // 2
                x1 = max(0, x - 25)
                y1 = max(0, y - 25)
                x2 = min(640, x + sz + 25)
                y2 = min(480, y + sz + 25)
                cv2.rectangle(arcuo_code_mask, (x1, y1), (x2, y2), (255, 255, 255), -1)
                color_image = cv2.bitwise_and(color_image, color_image, mask=arcuo_code_mask)

                cv2.imwrite("color_images\motion_blur_mask\motion_blur" + str(num_frames) + ".png", color_image)

        cv2.imshow("Output", color_image)
        key = cv2.waitKey(wait_key)

        # Press esc close the image window
        if key == 27:
            break

        # Press d to view the video frame by frame
        if key == ord('d'):
            if wait_key == 0:
                wait_key = 1
            else:
                wait_key = 0

        num_frames += 1
           
# Catch exception if the stream is ended
except RuntimeError:
    print("Stream ended")
        
finally:
    # Stop streaming
    cv2.destroyAllWindows()
    pipe.stop() """

In [ ]:
""" image = cv2.imread("color_images\motion_blur\motion_blur1229.png", cv2.IMREAD_GRAYSCALE)
corners, ids, rejected = detector.detectMarkers(image)
output_image = image.copy()
if corners is not None:
    print("Detected markers: ", len(corners))

cv2.aruco.drawDetectedMarkers(output_image, corners, ids)
cv2.imshow("Output", output_image)

# Show the marker id image
marker_id = ids[0][0]
print("Marker ID: ", marker_id)
marker_size = 200  # Taille de l'image générée
aruco_marker_image = cv2.aruco.generateImageMarker(aruco_dict, marker_id, marker_size)
cv2.imshow("Marker", aruco_marker_image)
cv2.waitKey(0)
cv2.destroyAllWindows() """